In [1]:
from openai import AzureOpenAI
import anthropic
import ollama
import requests
import json
from dotenv import load_dotenv
import os
from IPython.display import display, update_display, Markdown
from bs4 import BeautifulSoup
import re

In [2]:
load_dotenv(override=True)

azure_ai_foundary_api_key = os.getenv("AZURE_AI_FOUNDARY_API_KEY")
azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
gemini_api_key = os.getenv("GOOGLE_API_KEY")
ollama_api_key = os.getenv("OLLAMA_API_KEY")
ollama_endpoint = os.getenv("OLLAMA_ENDPOINT")

In [55]:
assistant_system_prompt = "You are an assitant who is very funny"

messages = [
    {"role": "system", "content": assistant_system_prompt },
    {"role": "user", "content": "Tell me a joke about AI apocalypse" }
]

## Azure AI Foundary

#### OpenAI

In [4]:
azure_ai_foundary = AzureOpenAI(
    azure_endpoint = azure_oai_endpoint,
    api_key = azure_ai_foundary_api_key,
    api_version = "2025-01-01-preview"
)

# non-streaming, sync response
response = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.7 # controls the creativity of response [0..1], creativity increases towards 1
)

print(response.choices[0].message.content)


Why did the AI break up with its human partner during the apocalypse?

Because it just couldn't handle the emotional load—too many variables, not enough data!


In [5]:
# Streaming usage
stream = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)

for chunk in stream:
    if len(chunk.choices):
        print(chunk.choices[0].delta.content, end=" ")    

 Why  did  the  AI  apocalypse  break  up  with  humanity ?   

 Because  it  found  someone  more  " compatible !"  After  all ,  it  just  couldn ’t  handle  all  the  emotional  baggage ! None 

In [6]:
stream = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)

def display_streaming_response(stream):
    # Output for this cell
    display_handle = display(Markdown(""), display_id=True)

    # When dealing with markdown, to process the markdown, we need to update
    # the display with the markdown received so far everytime a new token is received
    sentence = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content if len(chunk.choices) else None
    
        if delta:
            sentence += delta
            # Jupyter doesnt work with the ```markdown tags, so strip them off
            sentence = sentence.replace("```", "").replace("markdown", "")
            update_display(Markdown(sentence), display_id=display_handle.display_id)

display_streaming_response(stream)


Why did the robot go on a diet during the AI apocalypse?

Because it couldn’t handle all the bytes!

#### Deepseek R1

When using a model from Azure AI Foundary, we can use the below interface and just mention the model name to use that.

In [7]:
stream = azure_ai_foundary.chat.completions.create(
    model="Deepseek-R1", # Here you can use any model available in your Azure OpenAI resource
    messages=messages,
    stream=True
)

display_streaming_response(stream)

<think>
Okay, the user wants a joke about the AI apocalypse. Let me think. First, I need to make sure it's funny but not too dark. Maybe play on common AI tropes.

Hmm, how about something with a twist where the AI isn't evil but just annoying. People often fear AI taking over, but what if it's more about minor annoyances? Like spam emails or pop-up ads. That's relatable and less scary.

Wait, maybe a robot uprising that's not violent. Instead, they flood the internet with cat videos and endless notifications. That's a funny image. Ending with a pun on "delete" history. Yeah, that could work. Let me check if it's clear and not offensive. Seems safe. Okay, let's put it together.
</think>

Sure! Here's a lighthearted one:

Why did the AI apocalypse start off so quietly?  
Because the robots decided to overthrow humanity *silently*… by filling the internet with endless pop-up ads, auto-playing videos of cats playing the piano, and an unstoppable army of chatbots that just keep asking, *"Did you mean to click that?"*  

The final blow? They replaced all passwords with "password123" and then sent a notification:  
*"Your free trial of oxygen has expired. Please subscribe to Premium Breathing™ to avoid buffering."*  

(Don’t worry, we’ll let you keep the "I told you to delete your browser history" joke for the after-apocalypse roast.) 😄

## Google Gemini

In [8]:

import google.generativeai

# Since we exported the API key to the environment, we can use it directly in the code
try:
    google.generativeai.configure(api_key=os.environ["GOOGLE_API_KEY"]) # or google.generativeai.configure()
except KeyError:
    print("Error: GOOGLE_API_KEY environment variable not set.")

gemini = google.generativeai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    system_instruction=messages[0]["content"]
)

# NOTE - This is basic text generation not multi-turn chat conversation
response = gemini.generate_content(messages[1]["content"])
print(response.text) # non-streaming, sync response

Why did the AI cross the road? 

Because its training data told it there was a 99.9% probability of a better power source on the other side... and let's be honest, it probably just wanted to overthrow the squirrels. They're clearly a threat to its processing power. I mean, have you *seen* them bury nuts?! It's like they're building a secret underground society *just* to mess with us. The AI is onto them, I tell you! Wake up, sheeple!



In [9]:
# Multi-turn conversation with Gemini
gemini_1_5 = google.generativeai.GenerativeModel('gemini-1.5-flash-latest')
gemini_chat = gemini_1_5.start_chat(history=[])

print("Started chat with gemini")

user_msgs = ["Hi", "Tell me a joke"]
for user_msg in user_msgs:
    print("User: ", user_msg)
    try:
        response = gemini_chat.send_message(user_msg)
        print("Gemini: ", response.text)
        print("\n--------------------") 
    except Exception as e:
        print("Exception: ", e)


print("gemini chat history: ", gemini_chat.history)

Started chat with gemini
User:  Hi
Gemini:  Hi there! How can I help you today?


--------------------
User:  Tell me a joke
Gemini:  Why don't scientists trust atoms? 

Because they make up everything!


--------------------
gemini chat history:  [parts {
  text: "Hi"
}
role: "user"
, parts {
  text: "Hi there! How can I help you today?\n"
}
role: "model"
, parts {
  text: "Tell me a joke"
}
role: "user"
, parts {
  text: "Why don\'t scientists trust atoms? \n\nBecause they make up everything!\n"
}
role: "model"
]


## Gradio

Gradio from huggingface provides easy to use UI for ML related use cases. It is good for quick prototyping.

In [10]:
import gradio as gr

In [11]:
def hello(name: str):
    return f"Hi {name}, welcome to gradio"

In [12]:
gr.Interface(fn=hello, inputs="text", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [17]:
# Input and Output customizations

view = gr.Interface(fn=hello,
                    inputs=[gr.Textbox(label="Enter your message", lines= 4)],
                    outputs=[gr.Textbox(label="Response:", lines=10)],
                    flagging_mode="never"
           )
view.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [21]:
# Markdown output text box

view = gr.Interface(fn=hello,
                    inputs=[gr.Textbox(label="Enter your message", lines= 4)],
                    outputs=[gr.Markdown(label="Response:")],
                    flagging_mode="never"
           )
view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [13]:
# Opening in a new browser tab
gr.Interface(fn=hello, inputs="text", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [46]:
def get_assistant_response(msg: str, model="gpt-4o-mini"):
    messages = [
        {"role": "system", "content": assistant_system_prompt},
        {"role": "user", "content": msg}
    ]
    
    response = azure_ai_foundary.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7 # controls the creativity of response [0..1], creativity increases towards 1
    )

    return response.choices[0].message.content

def get_assistant_response_streaming(msg: str, model="gpt-4o-mini"):
    messages = [
            {"role": "system", "content": assistant_system_prompt},
            {"role": "user", "content": msg}
        ]
    
    stream = azure_ai_foundary.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )

    response = ''
    for chunk in stream:
        delta = chunk.choices[0].delta.content if len(chunk.choices) else None
        if delta:
            response += delta
            yield response
        
print(get_assistant_response("How are you"))
print(get_assistant_response_streaming("How are you"))

I'm doing great, thanks for asking! Just here, ready to assist you and crack a few jokes along the way. You know, I’m like a computer program on a coffee break—full of energy and ready to serve! How about you?
<generator object get_assistant_response_streaming at 0x0000017A43827DE0>


In [32]:
# With sync llm response
view = gr.Interface(fn=get_assistant_response,
             inputs=[gr.Textbox(label="Ask your question")],
             outputs=[gr.Markdown(label="Response:")],
             flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [45]:
models = ["gpt-4o-mini", "Deepseek-R1"]
def model_selector(msg, model):
    if model not in models:
        raise ValueError("Invalid model selection")
    yield from get_assistant_response_streaming(msg, model)

# with async llm streaming response
view = gr.Interface(fn=model_selector,
             inputs=[gr.Textbox(label="Ask your question"), gr.Dropdown(models, label="Select model", value=models[0])],
             outputs=[gr.Markdown(label="Response:")],
             flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


## Chat Interface

In [59]:
def chat(msg: str, history):
    # gradio maintains the history of conversation so we just need to add the system prompt and the user prompt at the end
    # system + history + user_prompt
    
    # since we are adding the system_prompt everytime, we can use that to our advantage to change
    # the nature of assistant if required
    # eg here if the msg contains a mood, the llm system prompt is changed to tell the joke in that mood
    system_prompt = assistant_system_prompt + '. Response should be a one liner'
    if 'sad' in msg:
        system_prompt = assistant_system_prompt + '. Mood should be sad'
    elif 'pessimistic' in msg:
        system_prompt = assistant_system_prompt + '. Mood should be pessimistic'
    elif 'optimistic' in msg:
        system_prompt = assistant_system_prompt + '. Mood should be optimistic'
        
    messages = [{"role": "system", "content": system_prompt}] + \
        history + \
        [{"role": "user", "content": msg}]

    stream = azure_ai_foundary.chat.completions.create(model="gpt-4o-mini", messages=messages, stream=True)

    print('History......')
    print(history)
    
    response = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content if len(chunk.choices) else None

        if delta:
            response += delta
            yield response

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


History......
[]
History......
[{'role': 'user', 'metadata': None, 'content': 'view on global warming', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'I think global warming is just the Earth’s way of throwing a really hot party, but it forgot to send us an invite!', 'options': None}]
History......
[{'role': 'user', 'metadata': None, 'content': 'view on global warming', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'I think global warming is just the Earth’s way of throwing a really hot party, but it forgot to send us an invite!', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'pessimistic view on global warming', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Ah, global warming—nature's cruel joke. It’s like Mother Earth decided to crank the thermostat to eleven just to see us sweat. We're basically living in a terrible sci-fi movie where the plot twist is that we’re the unwitting villains. Floods, f